In [ ]:
import pandas as pd

# Define file paths
content_path = r'C:\Users\User\Desktop\jupiter_files\Content.csv'
reactions_path = r'C:\Users\User\Desktop\jupiter_files\Reactions.csv'
reaction_types_path = r'C:\Users\User\Desktop\jupiter_files\ReactionTypes.csv'

# Load the datasets
df_content = pd.read_csv(content_path)
df_reactions = pd.read_csv(reactions_path)
df_reaction_types = pd.read_csv(reaction_types_path)

# Clean column names by stripping whitespace
df_content.columns = df_content.columns.str.strip()
df_reactions.columns = df_reactions.columns.str.strip()
df_reaction_types.columns = df_reaction_types.columns.str.strip()

# Clean 'Type' column values by stripping whitespace
df_content['Type'] = df_content['Type'].str.strip()
df_reactions['Type'] = df_reactions['Type'].str.strip()
df_reaction_types['Type'] = df_reaction_types['Type'].str.strip()

# Display cleaned column names
print("Cleaned Content DataFrame columns:")
print(df_content.columns)

print("\nCleaned Reactions DataFrame columns:")
print(df_reactions.columns)

print("\nCleaned ReactionTypes DataFrame columns:")
print(df_reaction_types.columns)

# Display unique values in 'Type' columns for each DataFrame after cleaning
print("\nCleaned Content DataFrame 'Type' column preview:")
print(df_content['Type'].unique())

print("\nCleaned Reactions DataFrame 'Type' column preview:")
print(df_reactions['Type'].unique())

print("\nCleaned ReactionTypes DataFrame 'Type' column preview:")
print(df_reaction_types['Type'].unique())

# Merge datasets on 'Content ID'
merged_df = pd.merge(df_content, df_reactions, on='Content ID', how='outer')

# Rename 'Type_x' and 'Type_y' columns
merged_df.rename(columns={'Type_x': 'Content_Type', 'Type_y': 'Reaction_Type'}, inplace=True)

# Display merged DataFrame columns to check new names
print("\nMerged DataFrame columns after renaming:")
print(merged_df.columns)

# Merge the result with reaction types on 'Reaction_Type'
try:
    final_df = pd.merge(merged_df, df_reaction_types, left_on='Reaction_Type', right_on='Type', how='left')
except KeyError as e:
    print(f"\nKeyError: {e}")
    print("\nMerged DataFrame 'Reaction_Type' column values:")
    print(merged_df['Reaction_Type'].unique())
    print("\nReactionTypes DataFrame 'Type' column values:")
    print(df_reaction_types['Type'].unique())

# Keep only the relevant columns
final_df = final_df[['Content ID', 'Content_Type', 'Category', 'Datetime', 'Sentiment', 'Score']]

# Save the final DataFrame to a new CSV file
final_output_path = r'C:\Users\User\Desktop\jupiter_files\final_output.csv'
final_df.to_csv(final_output_path, index=False)

print(f"\nFinal merged DataFrame saved to {final_output_path}")


In [ ]:
import pandas as pd

# Load the final output DataFrame
df_final = pd.read_csv(r'C:\Users\User\Desktop\jupiter_files\final_output.csv')

# Display initial columns and a few rows
print("Initial columns and a preview:")
print(df_final.head())
print(df_final.columns)

# 1. Remove rows with missing values in critical columns
# Identify critical columns based on the business questions
critical_columns = ['Content ID', 'Content_Type', 'Category', 'Datetime', 'Sentiment', 'Score']
df_final_clean = df_final.dropna(subset=critical_columns)

# 2. Change data types
# Convert 'Datetime' column to datetime format
df_final_clean.loc[:, 'Datetime'] = pd.to_datetime(df_final_clean['Datetime'], errors='coerce')

# Convert 'Content ID' to string (if not already)
df_final_clean.loc[:, 'Content ID'] = df_final_clean['Content ID'].astype(str)

# For 'Score', ensure it is numeric (useful for analysis)
df_final_clean.loc[:, 'Score'] = pd.to_numeric(df_final_clean['Score'], errors='coerce')

# 3. Remove irrelevant columns
# Keep only the relevant columns
df_final_clean = df_final_clean[['Content ID', 'Content_Type', 'Category', 'Datetime', 'Sentiment', 'Score']]

# Save the cleaned DataFrame to a new CSV file
df_final_clean.to_csv(r'C:\Users\User\Desktop\jupiter_files\final_output_cleaned.csv', index=False)

# Display cleaned DataFrame columns and a preview
print("\nCleaned columns and a preview:")
print(df_final_clean.head())
print(df_final_clean.columns)


In [ ]:
# Load the cleaned data
df_clean = pd.read_csv(r'C:\Users\User\Desktop\jupiter_files\final_output_cleaned.csv')

# Group by category and sum the scores
category_scores = df_clean.groupby('Category')['Score'].sum().reset_index()

# Sort and get top 5 categories
top_categories = category_scores.sort_values(by='Score', ascending=False).head(5)

# Save the top 5 categories
top_categories.to_csv(r'C:\Users\User\Desktop\jupiter_files\TopCategories.csv', index=False)

print("Top 5 categories saved to TopCategories.csv")


In [ ]:
import pandas as pd

# Define file paths
top_categories_path = r'C:\Users\User\Desktop\jupiter_files\TopCategories.csv'
final_output_cleaned_path = r'C:\Users\User\Desktop\jupiter_files\final_output_cleaned.csv'
combined_excel_path = r'C:\Users\User\Desktop\jupiter_files\final.xlsx'

# Load the CSV files into DataFrames
df_top_categories = pd.read_csv(top_categories_path)
df_final_clean = pd.read_csv(final_output_cleaned_path)

# Save both DataFrames to an Excel file with separate sheets
with pd.ExcelWriter(combined_excel_path, engine='openpyxl') as writer:
    df_top_categories.to_excel(writer, sheet_name='Top Categories', index=False)
    df_final_clean.to_excel(writer, sheet_name='Cleaned Data', index=False)

print(f"Data saved to {combined_excel_path}")

# Preview the saved Excel file
# You can use Excel to open and check the sheets, but here's how to read it back for verification
xls = pd.ExcelFile(combined_excel_path)

# Load and preview each sheet
df_top_categories_preview = pd.read_excel(xls, sheet_name='Top Categories')
df_final_clean_preview = pd.read_excel(xls, sheet_name='Cleaned Data')

print("\nPreview of 'Top Categories' sheet:")
print(df_top_categories_preview.head())

print("\nPreview of 'Cleaned Data' sheet:")
print(df_final_clean_preview.head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset from the specified sheets
file_path = r'C:\Users\User\Desktop\jupiter_files\final.xlsx'

# Read the Excel file
data_top_categories = pd.read_excel(file_path, sheet_name='Top Categories')
data_cleaned_data = pd.read_excel(file_path, sheet_name='Cleaned Data')

# Clean column names by stripping leading/trailing spaces
data_top_categories.columns = data_top_categories.columns.str.strip()
data_cleaned_data.columns = data_cleaned_data.columns.str.strip()

# Choose the appropriate dataset for analysis
# For this example, we'll assume you're analyzing 'Cleaned Data'
data = data_cleaned_data

# Check if the required columns exist
required_columns = ['Category', 'Score', 'Sentiment']
for column in required_columns:
    if column not in data.columns:
        raise ValueError(f"Required column '{column}' not found.")

# Remove duplicate categories and count unique categories
unique_categories_df = data['Category'].drop_duplicates().reset_index(drop=True)
unique_category_count = unique_categories_df.count()
print(f"Number of unique categories: {unique_category_count}")

# Prepare top 5 categories by aggregate score
top_5_categories = data.groupby('Category')['Score'].sum().sort_values(ascending=False).head(5)

# Add month column if 'Datetime' is available
if 'Datetime' in data.columns:
    data['Datetime'] = pd.to_datetime(data['Datetime'], errors='coerce')
    data['Month'] = data['Datetime'].dt.to_period('M')

    # Aggregate scores by category and month
    monthly_scores = data.groupby(['Category', 'Month'])['Score'].sum().unstack().fillna(0)

    # Heatmap for aggregate scores by category and month
    plt.figure(figsize=(12, 8))
    sns.heatmap(monthly_scores, cmap='viridis', annot=True, fmt=".1f", linewidths=0.5, linecolor='gray')
    plt.title('Aggregate Scores by Category and Month')
    plt.tight_layout()
    heatmap_path = r'C:\Users\User\Desktop\jupiter_files\heatmap_aggregate_scores.png'
    plt.savefig(heatmap_path)
    plt.show()

# Bar graph for unique categories count
plt.figure(figsize=(10, 6))
sns.barplot(x=['Unique Categories'], y=[unique_category_count], palette='coolwarm')
plt.ylabel('Count')
plt.title('Number of Unique Categories')
plt.xticks(rotation=45)
plt.tight_layout()
bar_chart_path = r'C:\Users\User\Desktop\jupiter_files\unique_categories_bar_graph.png'
plt.savefig(bar_chart_path)
plt.show()

# Pie chart for top 5 categories by aggregate score
plt.figure(figsize=(10, 8))
plt.pie(top_5_categories, labels=top_5_categories.index, autopct='%1.1f%%', startangle=140, 
        colors=sns.color_palette('plasma', len(top_5_categories)), wedgeprops=dict(width=0.4, edgecolor='w'))
plt.title('Top 5 Categories by Aggregate "Popularity" Score')
plt.tight_layout()
pie_chart_path = r'C:\Users\User\Desktop\jupiter_files\top_5_categories_pie_chart.png'
plt.savefig(pie_chart_path)
plt.show()

# Bar chart for sentiment distribution
plt.figure(figsize=(10, 6))
sentiment_counts = data['Sentiment'].value_counts()
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette='coolwarm')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Sentiment Distribution')
for i, count in enumerate(sentiment_counts):
    plt.text(i, count, str(count), ha='center', va='bottom')
plt.tight_layout()
sentiment_chart_path = r'C:\Users\User\Desktop\jupiter_files\sentiment_distribution_bar_chart.png'
plt.savefig(sentiment_chart_path)
plt.show()

# Bar chart for category distribution
plt.figure(figsize=(12, 8))
category_counts = data['Category'].value_counts()
sns.barplot(x=category_counts.index, y=category_counts.values, palette='magma')
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Category Distribution')
plt.xticks(rotation=45)
for i, count in enumerate(category_counts):
    plt.text(i, count, str(count), ha='center', va='bottom')
plt.tight_layout()
category_chart_path = r'C:\Users\User\Desktop\jupiter_files\category_distribution_bar_chart.png'
plt.savefig(category_chart_path)
plt.show()

# Most popular category based on the number of posts
most_popular_category = data['Category'].value_counts().idxmax()
print(f"Most popular category: {most_popular_category}")

# Count of reactions to the most popular category
reactions_to_most_popular = data[data['Category'] == most_popular_category].shape[0]
print(f"Reactions to the most popular category: {reactions_to_most_popular}")

# Key Insights
print("\nKey Insights:")
print(f"1. Number of unique categories: {unique_category_count}")
print(f"2. Most popular category: {most_popular_category}")
print(f"3. Count of reactions to the most popular category: {reactions_to_most_popular}")
print(f"4. Top 5 categories by aggregate score:\n{top_5_categories}")


In [ ]:
pip install python-pptx Pillow


In [ ]:
from pptx import Presentation
from pptx.util import Inches
from PIL import Image

# Create a presentation object
prs = Presentation()

# Paths to the saved charts
chart_paths = {
    "Unique Categories Bar Graph": r'C:\Users\User\Desktop\jupiter_files\unique_categories_bar_graph.png',
    "Top 5 Categories Pie Chart": r'C:\Users\User\Desktop\jupiter_files\top_5_categories_pie_chart.png',
    "Sentiment Distribution Bar Chart": r'C:\Users\User\Desktop\jupiter_files\sentiment_distribution_bar_chart.png',
    "Category Distribution Bar Chart": r'C:\Users\User\Desktop\jupiter_files\category_distribution_bar_chart.png',
    "Heatmap of Aggregate Scores": r'C:\Users\User\Desktop\jupiter_files\heatmap_aggregate_scores.png'
}

# Add a title slide
slide_title = prs.slides.add_slide(prs.slide_layouts[0])
title = slide_title.shapes.title
subtitle = slide_title.placeholders[1]
title.text = "Exploratory Data Analysis"
subtitle.text = "Key Visualizations and Insights"

# Function to add a slide with a chart image
def add_chart_slide(prs, title_text, image_path):
    slide_layout = prs.slide_layouts[5]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = title_text
    left = top = Inches(1)
    pic = slide.shapes.add_picture(image_path, left, top, width=Inches(8), height=Inches(5))

# Add slides for each chart
for title, path in chart_paths.items():
    add_chart_slide(prs, title, path)

# Add a slide for key insights
slide_insights = prs.slides.add_slide(prs.slide_layouts[1])
title = slide_insights.shapes.title
title.text = "Key Insights"

# Add text box for insights
insights_content = (
    "1. Number of unique categories: [Insert Number Here]\n"
    "2. Most popular category: [Insert Category Here]\n"
    "3. Count of reactions to the most popular category: [Insert Count Here]\n"
    "4. Top 5 categories by aggregate score:\n"
    "[Insert Top 5 Categories Here]"
)

# Define the position and size of the text box
left = Inches(1)
top = Inches(1.5)
width = Inches(8)
height = Inches(4.5)

text_box = slide_insights.shapes.add_textbox(left, top, width, height)
text_frame = text_box.text_frame
p = text_frame.add_paragraph()
p.text = insights_content
p.space_after = Inches(0.1)

# Save the presentation
pptx_file_path = r'C:\Users\User\Desktop\jupiter_files\EDA_Presentation.pptx'
prs.save(pptx_file_path)

print(f"Presentation saved at {pptx_file_path}")
